In [1]:
import pandas as pd
from tqdm import tqdm
import gensim
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# os.chdir(r"D://Proposal_Defense//Simulations")

In [5]:
def get_parent_dir(directory):
    import os
    return os.path.dirname(directory)

current_dirs_parent = get_parent_dir(os.getcwd())

In [6]:
os.chdir(current_dirs_parent)

In [7]:
os.getcwd()

'/lustre/work/statgrads/vpiyush2/MEnKF-ANN/Data_Generation'

In [8]:
from Utils.Script_utils import get_data_splits, first_LSTM_training, get_data_splits_new
from joblib import Parallel, delayed
import sys
from scipy.stats import norm

In [9]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 42

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# # 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")
# tf.config.set_visible_devices([], 'GPU')

from tqdm.notebook import tqdm
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
# from keras import backend as K
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# K.set_session(sess)

No GPU found


In [10]:
import matplotlib.pyplot as plt

import pickle

In [ ]:
# model_cbow = gensim.models.word2vec.Word2Vec.load(r"Data//word2vec_sg")

In [13]:
lr = 1e-3
# from sklearn.model_selection import train_test_split


## Read the data
data = pd.read_csv(r"Data//supervised.csv")

## We only using xylan and pectin
to_keep = ["pectin", "xylan"]

data = data[data["high_level_substr"].isin(to_keep)]

data = data.reset_index(drop = True)

features = [seq.replace("|", ",").replace(",", " ") for seq in data["sig_gene_seq"].values]

features  = np.array(features)

features = features.reshape(-1,1)

In [14]:
data["high_level_substr"].value_counts()

pectin    39
xylan     35
Name: high_level_substr, dtype: int64

In [15]:
data.head()

,PUL ID,high_level_substr,PULid,sig_gene_seq
0,PUL0014,pectin,PUL0014,"null,null,HTH_DeoR,HTH_AraC|HTH_AraC,8.A.59,PL..."
1,PUL0034,pectin,PUL0034,"CE10,1.B.35"
2,PUL0035,pectin,PUL0035,"3.A.1,3.A.1,3.A.1,3.A.1,PL2_2"
3,PUL0081,pectin,PUL0081,"MarR,null,null,AraC_binding,null,null,PfkB,nul..."
4,PUL0086,pectin,PUL0086,"PL10_1,CE8,PL11"


In [16]:
reps = 50

In [18]:
## generate 100 realizations of train valid and test
catch = Parallel(n_jobs=15, verbose = 10, backend = "threading")(delayed(get_data_splits_new)(data,  features,  i) for i in range(reps)) 

[Parallel(n_jobs=15)]: Using backend ThreadingBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=15)]: Done  11 tasks      | elapsed:    0.1s
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    0.1s
[Parallel(n_jobs=15)]: Done  27 out of  50 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=15)]: Done  33 out of  50 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=15)]: Done  39 out of  50 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=15)]: Done  45 out of  50 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=15)]: Done  50 out of  50 | elapsed:    0.2s finished


In [19]:
with open('Data//train_valid_test_splits_50.pkl', 'wb') as f:
    pickle.dump(catch, f)  